# Langevin dynamics

## Introduction

In this tutorial, you are going to learn about Langevin dynamics. It is a very easy and therefore widely used technique for two main reasons:
* As a thermostat. You can use Langevin dynamics to run the system at a certain temperature, including the expected fluctuations of the total energy.
* To include hydrodynamic interactions of suspended particles with the solvent.

In reality, you will always do both: Langevin thermostats will always include hydrodynamic friction.

Let's consider a single spherical colloidal particle in a fluid. Due to the absence of further particles and external fields, this particle experiences Brownian motion as a result of the interaction with the solvent molecules. While structural relaxation times for molecular fluids are of the order of $10^{-14}s$, relevant time scales for Brownian particles are known to be in the order of $10^{-9}s$. The distinction between slow and fast degrees of freedom allows to describe the motion of the colloidal particle in terms of the Langevin equation. This equation of motion describes the apparent random movement of the particle in the fluid and is given by 
\begin{equation}
    m\dot{{\bf v}}(t)=-\gamma {\bf v}(t)+{\bf f}(t).
    \tag{1}
\end{equation}
where $m$ denotes the particle mass and ${\bf v}(t)$ its velocity. Equation (1) arises from Newton's equation of motion considering that the interaction of the spherical Brownian particle with the solvent has two contributions: 1- a friction force, which is proportional to the velocity of the particle, for not too large velocities, with proportionality constant equal to the friction constant $\gamma$; and 2- a rapidly varying force ${\bf f}(t)$ with the time due to the the random collisions of the solvent molecules with the surface of the Brownian particle.

For a macroscopically large spherical particle, $\gamma$ is given by the Stokes' law
\begin{equation}
    \gamma = 6\pi\eta_0a,
\end{equation}
with $\eta_0$ the shear viscosity of the fluid and $a$ the radius of the Brownian particle. The ensemble average of the fluctuating force ${\bf f}(t)$ vanishes, 
\begin{equation}
    \langle {\bf f}(t)\rangle = 0,
\end{equation}
since the systematic interaction with the fluid is made explicit in the friction term. Owing to the separation in time scales, there is no correlation between impacts in any distinct time intervals. Thus, the second moments of ${\bf f}$ satisfy 
\begin{equation}
    \langle f_i(t)f_j(t')\rangle =2\gamma k_\text{B}T \delta_{i,j}\delta(t-t'),
\end{equation}
where one can see that the strength of the fluctuation force depends on the friction coefficient and the system temperature. 

Since only ensemble averaged properties of ${\bf f}$ are specified, it make no sense to look at a single deterministic solution of Eq. (1), but ensemble averaged quantities that characterize the dynamics of the spherical Brownian particle. The simplest quantity is the so-called mean square displacement (MSD) after time $\tau$
\begin{equation}
    \mathrm{MSD}(\tau)=\langle |{\bf r}(t+\tau)-{\bf r}(t)|^2\rangle.
\end{equation}
From integration of  Eq. (1) and considering that ${\bf v}(t)=\dot{{\bf r}}(t)$, one can obtain that
\begin{equation}
    \mathrm{MSD}(\tau)=6D\tau
\end{equation}
for $\tau\gg m/\gamma$, where the diffusion coefficient $D$ is defined as
\begin{equation}
    D=\frac{k_\text{B}T}{\gamma}.
\end{equation}

## 1. Setting up the observable

Write a function with signature `correlator_msd(pid, tau_max)` that returns a
mean-squared displacement correlator that is updated every time step. For help, you can refer to the documentation of [<tt>observables and correlators<tt>](https://espressomd.github.io/doc/analysis.html#observables-and-correlators).

```python
def correlator_msd(pids, tau_max):
    pos = espressomd.observables.ParticlePositions(ids=pids)
    pos_cor = espressomd.accumulators.Correlator(
        obs1=pos, tau_lin=16, tau_max=tau_max, delta_N=1,
        corr_operation="square_distance_componentwise", compress1="discard1")
    return pos_cor
```

## 2. Simulating Brownian motion

We will simulate the diffusion of a single particle that is coupled to an implicit solvent.

In [ ]:
import numpy as np
import logging
import sys

import espressomd
import espressomd.accumulators
import espressomd.observables

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

# Constants
KT = 1.1
STEPS = 400000

# System setup
system = espressomd.System(box_l=[16] * 3)
system.time_step = 0.01
system.cell_system.skin = 0.4

a_particle = system.part.add(pos=[0, 0, 0])

# Run for different friction coefficients
gammas = [1.0, 2.0, 4.0, 10.0]
tau_results = []
msd_results = []

for gamma in gammas:
    system.auto_update_accumulators.clear()
    system.thermostat.turn_off()
    system.thermostat.set_langevin(kT=KT, gamma=gamma, seed=42)

    logging.info("Equilibrating the system.")
    system.integrator.run(1000)
    logging.info("Equilibration finished.")

    # Setup observable correlator
    correlator = correlator_msd([a_particle.id], STEPS)
    system.auto_update_accumulators.add(correlator)

    logging.info("Sampling started for gamma = {}.".format(gamma))
    system.integrator.run(STEPS)
    correlator.finalize()
    tau_results.append(correlator.lag_times())
    msd_results.append(np.sum(correlator.result().reshape([-1, 3]), axis=1))

logging.info("Sampling finished.")

## 3. Data analysis
### 3.1 Plotting the results

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 22})

plt.figure(figsize=(10, 10))
plt.xlabel(r'$\tau$ [$\Delta t$]')
plt.ylabel(r'MSD [$\sigma^2$]')
for index, (tau, msd) in enumerate(zip(tau_results, msd_results)):
    # We skip the first entry since it's zero by definition and cannot be displayed
    # in a loglog plot. Furthermore, we only look at the first 100 entries due to
    # the high variance for larger lag times.
    plt.loglog(tau[1:100], msd[1:100], label=r'$\gamma=${:.1f}'.format(gammas[index]))
    plt.loglog(tau[1:100], 6*KT/gammas[index]*tau[1:100], "--", color="black", alpha=0.2, label="theory")
plt.legend()
plt.show()

### 3.2 Calculating the diffusion coefficient

In this script an implicit solvent and a single particle are created and thermalized.
The random forces on the particle will cause the particle to move.
The mean squared displacement is calculated during the simulation via a multiple-tau
correlator. 
Can you give an explanation for the quadratic time dependency for short times?

The MSD of a Brownian motion can be decomposed in three main regimes [1]:
* for short lag times $\tau < \tau_p$, the particle motion is not
  significantly impeded by solvent collisions: it's in the ballistic mode
  (collision-free regime) where $\operatorname{MSD}(t) \sim (k_BT / \gamma) t^2$
* for long lag times $\tau > \tau_f$, the particle motion is determined by
  numerous collisions with the solvent: it's in the diffusive mode where
  $\operatorname{MSD}(t) \sim 6t$
* for lag times between $\tau_p$ and $\tau_f$, there is a crossover mode

The values $\tau_p$ and $\tau_f$ can be obtained manually through visual
inspection of the MSD plot, or more accurately by non-linear fitting [2].

Here, we are interested in the diffusion constant. Hence, we can ignore the
ballistic regime and look at the diffusive regime in more detail.

Use the function [<tt>curve_fit()</tt>](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) from the module <tt>scipy.optimize</tt> to produce a fit for the linear regime and determine the diffusion coefficients for the different $\gamma$s.

For large $t$ the diffusion coefficient can be expressed as:

$$6D = \lim_{t\to\infty} \frac{\partial \operatorname{MSD}(t)}{\partial t}$$

which is simply the slope of the MSD in the diffusive mode.

In [ ]:
import scipy.optimize


def linear(x, a, b):
    return a * x + b


# cutoffs for the diffusive regime (different for each gamma value)
tau_f_values = [24, 22, 20, 17]
# cutoff for the data series (larger lag times have larger variance due to undersampling)
cutoff_limit = 90

diffusion_results = []

plt.figure(figsize=(10, 8))
plt.xlabel(r'$\tau$ [$\Delta t$]')
plt.ylabel(r'MSD [$\sigma^2$]')
for index, (tau_f, tau, msd) in enumerate(zip(tau_f_values, tau_results, msd_results)):
    (a, b), _ = scipy.optimize.curve_fit(linear, tau[tau_f:cutoff_limit], msd[tau_f:cutoff_limit])
    x = np.linspace(tau[tau_f], tau[cutoff_limit - 1], 50)
    p = plt.plot(x, linear(x, a, b), '-')
    plt.plot(tau[tau_f:cutoff_limit], msd[tau_f:cutoff_limit], 'o', color=p[0].get_color(),
             label=r'$\gamma=${:.1f}'.format(gammas[index]))
    diffusion_results.append(a / 6)

plt.legend()
plt.show()

Calculate the diffusion coefficient for all cases and plot them as a function of $\gamma$. What relation do you observe?

In the diffusive mode, one can derive $D = k_BT / \gamma$ from the Stokes–Einstein relation [3].

In [ ]:
plt.figure(figsize=(10, 8))
plt.xlabel(r'$\gamma$')
plt.ylabel('Diffusion coefficient [$\sigma^2/t$]')
x = np.linspace(0.9 * min(gammas), 1.1 * max(gammas), 50)
y = KT / x
plt.plot(x, y, '-', label=r'$k_BT\gamma^{-1}$')
plt.plot(gammas, diffusion_results, 'o', label='D')
plt.legend()
plt.show()

## References

[1] R. Huang, I. Chavez, K. Taute, et al. Direct observation of the full transition from ballistic to diffusive Brownian motion in a liquid. *Nature Phys.*, 7, 2011. doi:[10.1038/nphys1953](https://doi.org/10.1038/nphys1953)  
[2] M. K. Riahi, I. A. Qattan, J. Hassan, D. Homouz, Identifying short- and long-time modes of the mean-square displacement: An improved nonlinear fitting approach. *AIP Advances*, 9:055112, 2019. doi:[10.1063/1.5098051](https://doi.org/10.1063/1.5098051)  
[3] R. Huang, I. Chavez, K. Taute, et al. Direct observation of the full transition from ballistic to diffusive Brownian motion in a liquid. *Nature Phys.*, 7, 2011. doi:[10.1038/nphys1953](https://doi.org/10.1038/nphys1953)  